# Project 3: Predicting the Success of a Kickstarter Campaign
A supervised learning exercise featuring logistic regression, SVM, KNN, and [Others?]

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy
import os
import json

In [2]:
import sys
sys.executable

'/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/bin/python'

In [3]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

def to_str(obj):
    return str(obj).split(' ')[0]

def count_list_items(list):
    try:
        return len(list)
    except:
        return 0

In [4]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

In [5]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [52]:
# data at WebRobots https://webrobots.io/kickstarter-datasets/
csv_range = range(1, 49)

df = pd.DataFrame()
for csv in csv_range:
    csv = str(csv).zfill(3)
    path ='data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter{}.csv'.format(csv)
    frame = pd.read_csv(path)

    frame[csv] = csv
    df = df.append(frame, ignore_index=True)

In [53]:

# df48 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter048.csv')
# df47 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter047.csv')
# df46 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter046.csv')
# df45 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter045.csv')
# df44 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter044.csv')
# df43 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter043.csv')
# df42 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter042.csv')
# df41 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter041.csv')
# df40 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter040.csv')
# df39 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter039.csv')
# df38 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter038.csv')


# frames = [df38,df39,df40,df41,df42,df43,df44,df45,df46,df47,df48]

# df = pd.concat(frames)

print(df.shape)
print(df.columns)
# df.head()

(192716, 85)
Index(['001', '002', '003', '004', '005', '006', '007', '008', '009', '010',
       '011', '012', '013', '014', '015', '016', '017', '018', '019', '020',
       '021', '022', '023', '024', '025', '026', '027', '028', '029', '030',
       '031', '032', '033', '034', '035', '036', '037', '038', '039', '040',
       '041', '042', '043', '044', '045', '046', '047', '048', 'backers_count',
       'blurb', 'category', 'converted_pledged_amount', 'country',
       'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type'],
      dtype='object')


In [54]:
df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]

df['category_name'] = [json.loads(x)['name'] for x in df['category']]
df['creator_name'] = [json.loads(x)['name'] for x in df['creator']]
df['blurb_len2'] = df['blurb'].str.lower().str.split()
df['blurb_length'] = df['blurb_len2'].apply(count_list_items)

In [55]:
df['created_at'] = pd.to_datetime(df['created_at'],unit='s')
df['launched_at'] = pd.to_datetime(df['launched_at'],unit='s')
df['deadline'] = pd.to_datetime(df['deadline'],unit='s')

df['created'] = pd.to_datetime(df['created_at'])
df['launched'] = pd.to_datetime(df['launched_at'])
df['deadline'] = pd.to_datetime(df['deadline'])

df['created'] = df.created.apply(lambda x: x.date())
df['launched'] = df.launched.apply(lambda x: x.date())
df['deadline'] = df.deadline.apply(lambda x: x.date())

df.spotlight = df.spotlight.astype(int)
df.staff_pick = df.staff_pick.astype(int)


# print(df['launched'].iloc[0])

# df['created'] = df['created'].apply(lambda x: x.strftime('%Y-%m-%d'))
# df['launched'] = df['launched'].apply(lambda x: x.strftime('%Y-%m-%d'))
# df['deadline'] = df['deadline'].apply(lambda x: x.strftime('%Y-%m-%d'))

df['campaign_length'] = (df['deadline'] - df['launched']).apply(to_str)

df['pct_goal_achieved'] = round((df['usd_pledged'] / df['goal'])*100,1)
df.shape

(178076, 93)

In [56]:

df = df.sort_values(['deadline'], ascending=[False])
df.shape

(178076, 93)

In [57]:
df = df[['id','name','state','category_name','backers_count','pct_goal_achieved','usd_pledged','goal','country','currency','campaign_length',
         'deadline','launched','created','spotlight','staff_pick','creator_name','blurb_length']]
df = df.dropna()
print(df.shape)
print(df.columns)
df.head()

(178075, 18)
Index(['id', 'name', 'state', 'category_name', 'backers_count',
       'pct_goal_achieved', 'usd_pledged', 'goal', 'country', 'currency',
       'campaign_length', 'deadline', 'launched', 'created', 'spotlight',
       'staff_pick', 'creator_name', 'blurb_length'],
      dtype='object')


,id,name,state,category_name,backers_count,pct_goal_achieved,usd_pledged,goal,country,currency,campaign_length,deadline,launched,created,spotlight,staff_pick,creator_name,blurb_length
50474,1725323227,Inspire young girls,failed,Childrenswear,1,2.3,30.0,1300.0,US,USD,30,2018-01-12,2017-12-13,2017-12-08,0,0,Rayna,6
96852,2065169465,Cotton-Top Pastries,successful,Small Batch,99,131.4,9858.0,7500.0,US,USD,30,2018-01-12,2017-12-13,2017-12-12,1,1,Holly Weist,5
82814,1516902916,Dreaming Creek Brewery,failed,Drinks,64,30.7,6139.0,20000.0,US,USD,30,2018-01-12,2017-12-13,2017-08-11,0,0,Mike Bradley,19
97879,1396766240,Ripple Playing Cards - Printed by USPCC,failed,Playing Cards,131,33.9,3387.0,9999.0,US,USD,38,2018-01-12,2017-12-05,2017-10-08,0,0,B.Y. Eidelman,16
49869,1361347175,New Boutique Funding for the San Antonio Stock...,failed,Ready-to-wear,0,0.0,0.0,5000.0,US,USD,15,2018-01-12,2017-12-28,2017-12-22,0,0,Darrian Fosty,26


In [58]:
df.tail()

,id,name,state,category_name,backers_count,pct_goal_achieved,usd_pledged,goal,country,currency,campaign_length,deadline,launched,created,spotlight,staff_pick,creator_name,blurb_length
159484,266044220,Help me write my second novel.,successful,Fiction,18,112.6,563.0,500.0,US,USD,30,2009-05-29,2009-04-29,2009-04-29,1,1,Brendan Mackie,24
2373,199916122,Mr. Squiggles,failed,Illustration,0,0.0,0.0,30.0,US,USD,10,2009-05-22,2009-05-12,2009-05-12,0,0,C.K. Sample,22
109400,2089078683,New York Makes a Book!!,successful,Journalism,110,111.0,3329.0,3000.0,US,USD,18,2009-05-16,2009-04-28,2009-04-27,1,1,We Make a Book,22
6227,1703704063,drawing for dollars,successful,Illustration,3,175.0,35.0,20.0,US,USD,9,2009-05-03,2009-04-24,2009-04-24,1,1,darkpony,26
980,1703704063,drawing for dollars,successful,Illustration,3,175.0,35.0,20.0,US,USD,9,2009-05-03,2009-04-24,2009-04-24,1,1,darkpony,26


In [59]:
df.shape

(178075, 18)

In [60]:
df = df.drop_duplicates()
df.shape

(163426, 18)

In [61]:
# df['deadline'] = pd.to_datetime(df['deadline'])
# df['launched'] = pd.to_datetime(df['launched']) 
# df['launched'] = df['launched'].apply(lambda x: x.strftime('%Y-%m-%d'))
# df['launched'] = pd.to_datetime(df['launched']) 
# df = df.sort_values(['deadline'], ascending=[False])
# df['campaign_length'] = (df['deadline'] - df['launched']).apply(to_str)
# # df = df.set_index('ID')
# print(df.shape)
# df.head()

In [62]:
# # remove canceled, undefined, live and suspended
# df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]
# df = pd.DataFrame(df)
# start_date = pd.to_datetime('2012-01-01') 
# df = df[df['launched'] >= start_date] # filter from start date to current
# print(df.shape)
# df.tail()   

In [63]:
# # df['ID'] = df['ID'].apply(to_str)
# df['deadline'] = df['deadline'].apply(to_str)
# df['launched'] = df['launched'].apply(to_str)
# df['idx'] = df['ID']
# df = df.set_index('idx')

In [64]:
# Prep for categorical analysis
# df.main_category = pd.Categorical(df.main_category) 
# df['main_category_code'] = df.main_category.cat.codes

# df.country = pd.Categorical(df.country) 
# df['country_code'] = df.country.cat.codes

# df.currency = pd.Categorical(df.currency) 
# df['currency_code'] = df.currency.cat.codes

# df.state = pd.Categorical(df.state) 
# df['state_code'] = df.state.cat.codes

# df.head(100)

In [65]:
# df['main_category'] = pd.get_dummies('main_category')
# df.columns

In [66]:
# df_dummies = pd.get_dummies(df[['state','main_category','country','currency']],drop_first=True)
# # df['state'] = pd.get_dummies(df['state'])
# # df['main_category'] = pd.get_dummies(df['main_category'])
# # df['country'] = pd.get_dummies(df['country'])
# # df['currency'] = pd.get_dummies(df['currency'])
# df['ID'].head()

In [67]:
# df = df_dummies.merge(df,how='inner',left_index=True, right_index=True)

# df.columns

In [68]:
# df.info()

In [ ]:
# df.head()

In [ ]:
df.to_pickle('data/kickstarter_data_ds2.pkl')
try:
    df.to_sql("kickstarter_data_ds2", engine, chunksize=20000, if_exists='replace')
except Exception as e:
    print(e)
    pass

# Temporary fix:

# df.to_csv('data/kickstarter_data.csv')

## State of Success [this should now be covered in EDA]

In [ ]:
# df_state = df.groupby(["state"]).count().sort_values(['name'], ascending=[False])
# df_state = pd.DataFrame(df_state, columns={'name'})
# print(df_state.shape)
# df_state

In [ ]:
# df_state = df.groupby(["state"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_state = pd.DataFrame(df_state, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_state.shape)
# df_state

In [ ]:
# df_country = df.groupby(["country"]).count().sort_values(['name'], ascending=[False])
# df_country = pd.DataFrame(df_country, columns={'name'})
# print(df_country.shape)
# df_country

In [ ]:
# df_country = df.groupby(["country"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_country = pd.DataFrame(df_country, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_country.shape)
# df_country

In [ ]:
# df_currency = df.groupby(["currency"]).count().sort_values(['name'], ascending=[False])
# df_currency = pd.DataFrame(df_currency, columns={'name'})
# print(df_currency.shape)
# df_currency

In [ ]:
# df_currency = df.groupby(["currency"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_currency = pd.DataFrame(df_currency, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_currency.shape)
# df_currency

In [ ]:
# df_main_category = df.groupby(["main_category"]).count().sort_values(['name'], ascending=[False])
# df_main_category = pd.DataFrame(df_main_category, columns={'name'})
# print(df_main_category.shape)
# df_main_category

In [ ]:
# df_main_category = df.groupby(["main_category"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_main_category = pd.DataFrame(df_main_category, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_main_category.shape)
# df_main_category

In [ ]:
# df_category = df.groupby(["category"]).count().sort_values(['name'], ascending=[False])
# print(df_category.shape)
# df_category

In [ ]:
# df_category = df.groupby(["category"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_category = pd.DataFrame(df_category, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_category.shape)
# df_category

In [ ]:
# df2 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter048.csv')
# df2.head()